In [1]:
from datasets import Dataset, DatasetDict

In [2]:
dataset = Dataset.from_csv('/mnt/data/users/keller/ROCStories/ROCStories_winter2017 - ROCStories_winter2017.csv')
dataset

Using custom data configuration default-ab73c4f3571a1168


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /mnt/data/users/keller/.cache/csv/default-ab73c4f3571a1168/0.0.0. Subsequent calls will reuse this data.


Dataset({
    features: ['storyid', 'storytitle', 'sentence1', 'sentence2', 'sentence3', 'sentence4', 'sentence5'],
    num_rows: 52665
})

In [3]:
len(dataset)

52665

In [4]:
from random import shuffle
from random import seed as set_seed

def make_shuffle_func(sep_token):
    def shuffle_stories(entries, seed=42):
        set_seed(seed)
        entries_as_dicts = [dict(zip(entries, values)) for values in zip(*entries.values())]
        converted_entries = []
        for entry in entries_as_dicts:
            sents = [entry[key] for key in sorted([key for key in entry.keys() if key.startswith('sentence')], key=lambda x: int(x[-1]))]
            sent_idx = list(range(len(sents)))
            sents_with_idx = list(zip(sents, sent_idx))
            shuffle(sents_with_idx)
            text = f'{sep_token} ' + f' {sep_token} '.join([s[0] for s in sents_with_idx]) 
            so_targets = [s[1] for s in sents_with_idx]
            shuffled_entry = {'text': text, 'so_targets': so_targets}
            converted_entries.append(shuffled_entry)
        new_entry = {key: [entry[key] for entry in converted_entries] for key in converted_entries[0]}
        return new_entry
    return shuffle_stories

In [5]:
map_func = make_shuffle_func('[CLS]')

In [6]:
dataset = dataset.map(map_func, batched=True)

  0%|          | 0/53 [00:00<?, ?ba/s]

In [7]:
dataset

Dataset({
    features: ['sentence1', 'sentence2', 'sentence3', 'sentence4', 'sentence5', 'so_targets', 'storyid', 'storytitle', 'text'],
    num_rows: 52665
})

In [8]:
train_test = dataset.train_test_split(test_size=0.2, seed=42)

test_validation = train_test['test'].train_test_split(test_size=0.3, seed=42)

dataset = DatasetDict({
    'train': train_test['train'],
    'test': test_validation['train'],
    'val': test_validation['test']})

In [9]:
dataset.save_to_disk('rocstories')
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'sentence3', 'sentence4', 'sentence5', 'so_targets', 'storyid', 'storytitle', 'text'],
        num_rows: 42132
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'sentence3', 'sentence4', 'sentence5', 'so_targets', 'storyid', 'storytitle', 'text'],
        num_rows: 7373
    })
    val: Dataset({
        features: ['sentence1', 'sentence2', 'sentence3', 'sentence4', 'sentence5', 'so_targets', 'storyid', 'storytitle', 'text'],
        num_rows: 3160
    })
})

In [10]:
for entry in dataset['train']:
    print(entry['text'])
    break

[CLS] They got hung up on a rock near the end. [CLS] They had a good time and pledged to go again. [CLS] The current was very fast through the shoals. [CLS] Bob took his daughter canoeing on the river. [CLS] The entire trip was about three miles.
